# HHL algorithm

In [6]:
from qiskit import QuantumRegister, QuantumCircuit
import numpy as np
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit import IBMQ, transpile

# for mitagating readout errors
from qiskit.utils.mitigation import complete_meas_cal


In [7]:
def HHL_2(diag: float, offdiag: float, theta: float, t: float):

    NUM_QUBITS = 4  # Total number of qubits
    nb = 1  # Number of qubits representing the solution
    nl = 2  # Number of qubits representing the eigenvalues

    a = diag  # Matrix diagonal
    b = offdiag  # Matrix off-diagonal

    # Initialize the quantum and classical registers
    qr = QuantumRegister(NUM_QUBITS)

    # Create a Quantum Circuit
    qc = QuantumCircuit(qr)

    qrb = qr[0:nb]
    qrl = qr[nb:nb+nl]
    qra = qr[nb+nl:nb+nl+1]

    # State preparation.
    qc.ry(2*theta, qrb[0])

    # QPE with e^{iAt}
    for qu in qrl:
        qc.h(qu)

    qc.p(a*t, qrl[0])
    qc.p(a*t*2, qrl[1])

    qc.u(b*t, -np.pi/2, np.pi/2, qrb[0])


    # Controlled e^{iAt} on \lambda_{1}:
    params=b*t

    qc.p(np.pi/2,qrb[0])
    qc.cx(qrl[0],qrb[0])
    qc.ry(params,qrb[0])
    qc.cx(qrl[0],qrb[0])
    qc.ry(-params,qrb[0])
    qc.p(3*np.pi/2,qrb[0])

    # Controlled e^{2iAt} on \lambda_{2}:
    params = b*t*2

    qc.p(np.pi/2,qrb[0])
    qc.cx(qrl[1],qrb[0])
    qc.ry(params,qrb[0])
    qc.cx(qrl[1],qrb[0])
    qc.ry(-params,qrb[0])
    qc.p(3*np.pi/2,qrb[0])

    # Inverse QFT
    qc.h(qrl[1])
    qc.rz(-np.pi/4,qrl[1])
    qc.cx(qrl[0],qrl[1])
    qc.rz(np.pi/4,qrl[1])
    qc.cx(qrl[0],qrl[1])
    qc.rz(-np.pi/4,qrl[0])
    qc.h(qrl[0])

    # Eigenvalue rotation
    t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
    t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
    t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
    t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

    qc.cx(qrl[1],qra[0])
    qc.ry(t1,qra[0])
    qc.cx(qrl[0],qra[0])
    qc.ry(t2,qra[0])
    qc.cx(qrl[1],qra[0])
    qc.ry(t3,qra[0])
    qc.cx(qrl[0],qra[0])
    qc.ry(t4,qra[0])
    qc.measure_all()

    return qc

    # print(f"Depth: {qc.depth()}")
    # print(f"CNOTS: {qc.count_ops()['cx']}")
    # qc.draw(fold=-1)

In [12]:
def extract_counts(dict):
    total_counts = 0
    relevant_count0 = 0
    relevant_count1 = 0
    for key in dict:
        if key[0] == '1':
            if key[-1] == '1':
                relevant_count1 += dict[key]
            if key[-1] == "0":
                relevant_count0 += dict[key]
        total_counts += dict[key]
    return (relevant_count0/total_counts, relevant_count1/total_counts)

In [9]:
def extract_solution(probs):
    return (np.sqrt(probs[0]), np.sqrt(probs[1]))

In [10]:
def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

## Simulations

In [14]:
def paper_no2():
    simulator = QasmSimulator()
    qc2 = HHL_2(1.5, 0.5, np.pi/2, 1)
    layout = [2,3,0,4]

    compiled_circuit = transpile(qc2, backend=simulator, initial_layout=layout)
    chip_qubits = 5
    meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                                qr=QuantumRegister(chip_qubits))
    qcs2 = meas_cals + [compiled_circuit]
    job2 = simulator.run(qcs2, shots=1024)

    result2=job2.result()
    counts2=result2.get_counts()
    p2 = extract_counts(counts2[-1])
    sol2 = extract_solution(p2)

    classical_sol = (0.354, 0.354)

    f2 = calculate_normalized_fidelity(classical_sol, sol2)

    print(p2, sol2, f2)
    print(counts2[-1])
paper_no2()

(0.18359375, 0.572265625) (0.42847841252506524, 0.7564824023068878) 0.9637590941974993
{'1011': 7, '1010': 15, '0110': 26, '0011': 17, '0001': 33, '1100': 170, '1110': 3, '0111': 15, '0000': 94, '0010': 65, '1111': 4, '1101': 575}


In [13]:
def paper_no4():
    simulator = QasmSimulator()
    qc2 = HHL_2(1.5, 0.5, -2.23, 1)
    layout = [2,3,0,4]

    compiled_circuit = transpile(qc2, backend=simulator, initial_layout=layout)
    chip_qubits = 5
    meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                                qr=QuantumRegister(chip_qubits))
    qcs2 = meas_cals + [compiled_circuit]
    job2 = simulator.run(qcs2, shots=1024)

    result2=job2.result()
    counts2=result2.get_counts()
    p2 = extract_counts(counts2[-1])
    sol2 = extract_solution(p2)

    classical_sol = (0.555, 0.784)

    f2 = calculate_normalized_fidelity(classical_sol, sol2)

    print(p2, sol2, f2)
    print(counts2[-1])
paper_no4()

(0.3564453125, 0.462890625) (0.5970304117044625, 0.6803606580336637) 0.9945717224264183
{'1110': 2, '1010': 20, '1011': 10, '0011': 43, '0001': 9, '0110': 27, '1100': 343, '1101': 462, '1111': 2, '0111': 10, '0000': 36, '0010': 60}


## On Quantum Hardware

In [19]:

# load IBM account
IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
IBMQ.load_account()

provider = IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-education', group='harvard', project='qse-210')


ibmqfactory.load_account:WARNING:2023-04-23 16:05:59,172: Credentials are already in use. The existing account in the session will be replaced.


In [22]:
# backend=provider.get_backend("ibm_perth")
backend=provider.get_backend("ibmq_quito")

In [24]:
def paper_no2_hardware():
    qc2 = HHL_2(1.5, 0.5, np.pi/2, 1)
    layout = [0,1,2,3]

    compiled_circuit = transpile(qc2, backend=backend, initial_layout=layout)
    chip_qubits = 5
    meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                                qr=QuantumRegister(chip_qubits))
    qcs2 = meas_cals + [compiled_circuit]
    job2 = backend.run(qcs2, shots=8192)

    result2=job2.result()
    counts2=result2.get_counts()
    p2 = extract_counts(counts2[-1])
    sol2 = extract_solution(p2)

    classical_sol = (0.354, 0.354)

    f2 = calculate_normalized_fidelity(classical_sol, sol2)

    print(p2, sol2, f2)
    print(counts2[-1])
paper_no2_hardware()

(0.227294921875, 0.327392578125) (0.4767545719497612, 0.5721822944875173) 0.9958872269528781
{'0000': 892, '0001': 444, '0010': 518, '0011': 298, '0100': 149, '0101': 195, '0110': 575, '0111': 577, '1000': 191, '1001': 150, '1010': 238, '1011': 155, '1100': 1128, '1101': 2158, '1110': 305, '1111': 219}


AttributeError: 'str' object has no attribute 'status'